In [1]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests

In [2]:
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.1
Default GPU Device: /gpu:0


In [3]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'

    # Load the vgg model from thr given path
    tf.saved_model.loader.load(sess, [vgg_tag],vgg_path)

    # extract the vgg layers
    vgg_input = tf.get_default_graph().get_tensor_by_name(vgg_input_tensor_name)
    keep_prob = tf.get_default_graph().get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3_out = tf.get_default_graph().get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4_out = tf.get_default_graph().get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7_out = tf.get_default_graph().get_tensor_by_name(vgg_layer7_out_tensor_name)

    return vgg_input, keep_prob, layer3_out, layer4_out, layer7_out

tests.test_load_vgg(load_vgg, tf)

Tests Passed


In [4]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer7_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer3_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    #return None  
    #resampling vgg_layer7 from 5x18x4096 to 5x18x2
    def conv1x1(x):
        return tf.layers.conv2d(x, filters=num_classes, kernel_size = 1, padding='same',
                                kernel_regularizer = tf.contrib.layers.l2_regularizer(0.001), 
                                kernel_initializer = tf.truncated_normal_initializer(stddev=0.01))
    
    def conv2d_transpose(x, kernel_size, strides):
        return tf.layers.conv2d_transpose(x, filters = num_classes, kernel_size = kernel_size,
                                          strides = strides, padding = 'same',
                                          kernel_regularizer = tf.contrib.layers.l2_regularizer(0.001),
                                          kernel_initializer = tf.truncated_normal_initializer(stddev=0.01))
    
    resampled_vgg7 = conv1x1(vgg_layer7_out)
    
    #upsampling vgg7 from 5x18x2 to 10x36x2
    upsampled_vgg7 = conv2d_transpose(resampled_vgg7, 4, 2)
    
    #resampling vgg_layer4 from 10x36x512 to 10x36x2
    resampled_vgg4 = conv1x1(vgg_layer4_out)
    
    #skip_connections 
    skip_layer = tf.add(upsampled_vgg7, resampled_vgg4)
    
    #upsampling skip layer from 10x36x2 to 20x72x2
    upsampled_skip_layer = conv2d_transpose(skip_layer, 4, 2)
    
    #resample vgg3 from 20x72x256 to 20x72x2
    resampled_vgg3 = conv1x1(vgg_layer3_out)
    
    #skip connections
    skip_layer2 = tf.add(upsampled_skip_layer, resampled_vgg3)
    
    #upsample skip layer from 20x72x2 to 160x576x2
    upsampled_skip_layer2 = conv2d_transpose(skip_layer2, 16, 8)
    
    return upsampled_skip_layer2

    
tests.test_layers(layers)

Tests Passed


In [5]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    
#    return None, None, None
    
    # Name logits Tensor, so that is can be loaded from disk after training
    #logits = tf.identity(logits, name='logits')
    
    #the output tensor is 4D so we have to reshape it to 2D
    #logits is now a 2D tensor where each row represents a pixel and each column a class
    logits = tf.reshape(nn_last_layer, (-1, num_classes))

#    iou, iou_op = tf.metrics.mean_iou(correct_label, prediction, num_classes)
    # Loss and Optimizer
#    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=correct_label))
#    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= logits, labels= correct_label))
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy_loss)
    
    
    return logits, optimizer, cross_entropy_loss
    
tests.test_optimize(optimize)

Tests Passed


In [6]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    # TODO: Implement function
    sess.run(tf.global_variables_initializer())
    
    for e in range(epochs):
        print("Number of Epochs", e)
        for batch_image, batch_label in get_batches_fn(batch_size):
            _ = sess.run(train_op, feed_dict = {correct_label : batch_label, input_image: batch_image, learning_rate:0.1, keep_prob:1.0})
            loss = sess.run(cross_entropy_loss, feed_dict = {correct_label : batch_label, input_image: batch_image, learning_rate:0.1, keep_prob:1.0})
            print("Loss: = {:.3f}".format(loss))
            #pass
    #for epochs in epochs:
    #    for batch_image, batch_label in get_batches_fn(batch_size):
        #    #log, opt, cross_entropy = optimize(nn_last_layer, batch_label, learning_rate, num_classes=2)
        #    _ , loss = sess.run(train_op, cross_entropy_loss, feed_dict = {correct_label : batch_label, input_image: batch_image, learning_rate:0.1, keep_prob:1.0} )
    #        print(1)
    #pass
tests.test_train_nn(train_nn)

In [7]:
def run():
    num_classes = 2
    image_shape = (160, 576)
    #data_dir = './data'
    data_dir = '/input'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        # TODO: Build NN using load_vgg, layers, and optimize function
        image_input, keep_prob, vgg_layer3_out, vgg_layer4_out, vgg_layer7_out = load_vgg(sess, vgg_path)
        upsampled_skip_layer2 = layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes)
        
        #
        #defining placeholders
        #input_image = tf.placeholder(tf.float32, shape = [None, None, None, None])
        learning_rate =  tf.placeholder(tf.float32, shape=[])
        #keep_prob = tf.placeholder(tf.float32, shape=[])
        correct_label = tf.placeholder(tf.float32, shape = [None, None, None, num_classes])
        
        logits, optimizer, cross_entropy_loss = optimize(upsampled_skip_layer2, correct_label, learning_rate, num_classes)
        

        # TODO: Train NN using the train_nn function
        train_nn(sess, 1, 1, get_batches_fn, optimizer, cross_entropy_loss, image_input,
             correct_label, keep_prob, learning_rate)
        

        # TODO: Save inference data using helper.save_inference_samples
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, image_input)

        # OPTIONAL: Apply the trained model to a video


if __name__ == '__main__':
    run()